In [1]:
2+2

4

In [53]:
# profile = cancun_variables.profile_report(title='Pandas Profiling Report')
# profile.to_file(output_file="cancun_variables.html")


In [ ]:
# IDEA: We currently have the issue where we cannot benchmark our model. Perhaps to prove that it is better than random selection,
# scrape the 1st page of each price_range and see how much of our TOP 50 listings overlap with the 1st page Airbnb Listing Rankings,
# compare to a random selection of listings, bootstrap the proportion/median/difference and see whether we have a significance difference.

# Null Hypothesis: Our model will have the same overlap proportion with Airbnb Ranking (1st page) as a random selection
# Alternative Hypothesis: Our model will have more first page listings than a random selection

# This all depends on Airbnb's ranking algorithm. I wonder how it works! 

In [54]:
# TODO: Variable section within variables. Try to filter down to ~10 to work with